In [7]:
import pandas as pd

df_train = pd.read_csv('../data/raw/train.csv')

In [8]:
import optuna
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Replace with your actual path and filename
df_train = pd.read_csv('../data/raw/train.csv')

# Separate features (X) and target (y)
# Replace 'target' with the name of your actual target column
target = 'efs'
X = df_train.drop([target, 'efs_time'], axis=1)
X = pd.get_dummies(X, dummy_na=True)
y = df_train[target]


def objective(trial):
    # --- 3.1. Suggest values of hyperparameters from Optuna ---
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'random_strength': trial.suggest_float('random_strength', 1e-2, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 256),
        'verbose': 0,                # Suppress CatBoost training output
        'random_state': 42           # Ensures reproducibility
    }

    # --- 3.2. Define the model ---
    model = CatBoostClassifier(**param)

    # --- 3.3. Evaluate using cross-validation ---
    # Using StratifiedKFold for classification
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Use scoring='roc_auc' for AUC; you can change this to 'accuracy', etc.
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)

    # --- 3.4. The objective to maximize (Optuna will try to find best hyperparams for the highest AUC) ---
    return cv_scores.mean()

# Create study - we want to maximize the AUC
study = optuna.create_study(direction='maximize')

# Run optimization. You can adjust n_trials to fit your needs.
study.optimize(objective, n_trials=7, show_progress_bar=True)

print("Number of finished trials:", len(study.trials))
print("Best trial:")
trial_ = study.best_trial

print("  Value (AUC):", trial_.value)
print("  Params: ")
for key, value in trial_.params.items():
    print(f"    {key}: {value}")


best_params = study.best_params

# You may want to set verbose=100 if you want to monitor training logs
# or keep it at 0 (silent) for no logs
final_model = CatBoostClassifier(
    **best_params,
    verbose=100,         # or 0
    random_state=42
)

final_model.fit(X, y)

[I 2025-01-31 15:24:19,447] A new study created in memory with name: no-name-0c13e386-c93d-4387-a465-34d60d8a265d


  0%|          | 0/7 [00:00<?, ?it/s]

[I 2025-01-31 15:24:26,030] Trial 0 finished with value: 0.7544862971879309 and parameters: {'iterations': 267, 'depth': 6, 'learning_rate': 0.11546554311730166, 'l2_leaf_reg': 5.180150082882407, 'random_strength': 0.1542968575797714, 'bagging_temperature': 0.8506264893606051, 'border_count': 129}. Best is trial 0 with value: 0.7544862971879309.
[I 2025-01-31 15:24:53,201] Trial 1 finished with value: 0.7373486447647382 and parameters: {'iterations': 600, 'depth': 10, 'learning_rate': 0.11117648956543628, 'l2_leaf_reg': 3.292913821783733, 'random_strength': 3.6225081709504887, 'bagging_temperature': 0.03096290274311675, 'border_count': 218}. Best is trial 0 with value: 0.7544862971879309.
[I 2025-01-31 15:24:57,849] Trial 2 finished with value: 0.7067611543781374 and parameters: {'iterations': 359, 'depth': 4, 'learning_rate': 0.0018352530407874472, 'l2_leaf_reg': 6.3605961594870015, 'random_strength': 7.530839233086073, 'bagging_temperature': 0.3326336532748174, 'border_count': 80}. B

In [9]:
# # save the model
# import joblib
# joblib.dump(final_model, '../models/catboost_model.pkl')

In [10]:
df_test = pd.read_csv('../data/raw/test.csv')

In [11]:
# make predictions
X_test = pd.get_dummies(df_test, dummy_na=True)
y_pred = final_model.predict(X_test)

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 23 should be feature with name dri_score_High (found dri_score_Intermediate).